In [18]:
import simpy
import random
import numpy as np

simulation_time = 7*60*60
cola = 0
cola_max = 0
tiempo_ultima_atencion = 0
tiempo_maximo_espera = 0


def getLambda(env):
    if(env.now <= 2*60*60):
        return 1/240
    elif(env.now <= 5*60*60):
        return 1/120
    elif(env.now <= 7*60*60):
        return 1/360
    
    
def duracionAtencion():
    u = np.random.uniform()
    if(u <= 0.1):
        return np.random.uniform(1,7) * 60
    elif(u <= 0.8):
        return np.random.uniform(1,3) * 60
    return np.random.uniform(1,5) * 60
    
def cajero(env):
    global cola
    index = 1
    atm = simpy.Resource(env, capacity=1)
    while True:
        arriboClientes = random.expovariate(getLambda(env))
        yield env.timeout(arriboClientes)
        print('llega un cliente')
        cola +=1
        print('cola en el cajero', cola)
        env.process(atencion(env, index, atm))
        index +=1

def atencion(env, index, atm):
    global cola, cola_max, tiempo_ultima_atencion, tiempo_maximo_espera
    request = atm.request()
    yield request
    duracion =  duracionAtencion()
    yield env.timeout(duracion)
    atm.release(request)
    print('Uso el cajero')
    
    #Chuequeo la cola maxima
    if cola > cola_max: cola_max = cola
    
    print('cola antes de atender al cliente:', cola)
    cola -= 1
    print("Cola despues de la atencion:", cola)
    
    #Chequeo la cantidad maxima de espera
    if (round(env.now/60, 2) - tiempo_ultima_atencion > tiempo_maximo_espera):
        tiempo_maximo_espera = round(env.now/60, 2) - tiempo_ultima_atencion
        tiempo_ultima_atencion = round(env.now/60, 2)
    
environment = simpy.Environment()
environment.process(cajero(environment))
environment.run(until=simulation_time)

print('\nLa cola maxima que se formo fue de ' + str(cola_max) + ' personas.')
print('La cantidad maxima de espera de un cliente fue ' + str(round(tiempo_maximo_espera, 2)) + ' minutos.')

llega un cliente
cola en el cajero 1
llega un cliente
cola en el cajero 2
Uso el cajero
cola antes de atender al cliente: 2
Cola despues de la atencion: 1
Uso el cajero
cola antes de atender al cliente: 1
Cola despues de la atencion: 0
llega un cliente
cola en el cajero 1
Uso el cajero
cola antes de atender al cliente: 1
Cola despues de la atencion: 0
llega un cliente
cola en el cajero 1
Uso el cajero
cola antes de atender al cliente: 1
Cola despues de la atencion: 0
llega un cliente
cola en el cajero 1
Uso el cajero
cola antes de atender al cliente: 1
Cola despues de la atencion: 0
llega un cliente
cola en el cajero 1
Uso el cajero
cola antes de atender al cliente: 1
Cola despues de la atencion: 0
llega un cliente
cola en el cajero 1
Uso el cajero
cola antes de atender al cliente: 1
Cola despues de la atencion: 0
llega un cliente
cola en el cajero 1
llega un cliente
cola en el cajero 2
llega un cliente
cola en el cajero 3
Uso el cajero
cola antes de atender al cliente: 3
Cola despues 